In [1]:
import sqlite3
import pandas as pd

## View Dataset-ships

In [3]:
conn = sqlite3.connect('data/Dataset-ships.db')

query_arenas = "SELECT * FROM arenas"
query_members = "SELECT * FROM arena_members"
query_glossary = "SELECT * FROM glossary_ships"
query_catalog = "SELECT * FROM catalog_items"

df_arenas = pd.read_sql_query(query_arenas, conn)
df_members = pd.read_sql_query(query_members, conn)
df_glossary = pd.read_sql_query(query_glossary, conn)
df_catalog = pd.read_sql_query(query_catalog, conn)

conn.close()

In [24]:
df_arenas.head()

,arena_id,periphery_id,winner_team_id,start_dt,duration_sec,map_type_id,team_build_type_id,battle_level_id
0,1831461459124775,4624,0,2017-09-07 06:07:35,1133,51,0,8
1,6293769271722567,4622,0,2017-09-16 15:24:55,18,2,1,-1
2,3139493749735121,4622,0,2017-09-14 08:33:21,807,9,0,8
3,1989322983765984,4624,1,2017-09-26 15:53:04,396,24,2,3
4,2915949292071084,4622,0,2017-09-16 04:29:32,1041,51,0,8


In [25]:
df_arenas.shape

(100000, 8)

In [20]:
df_members.head()

,arena_id,periphery_id,account_db_id,team_id,clan_db_id,vehicle_type_id,ships_killed,planes_killed,damage,team_damage,received_damage,regen_hp,max_hp,is_alive,distance,credits,exp
0,73990908078847,4624,2016609712,0,0,4289640432,2,0,42635,0,26801,0,26800,0,19.335600,46215,218
1,73990908078847,4624,-269540015,1,0,4280170480,1,0,11274,0,13100,0,13100,0,11.285300,30620,263
2,73990908078847,4624,-269540014,1,0,4289640144,0,0,1089,0,0,0,28300,1,30.564699,16436,227
3,73990908078847,4624,-269540013,1,0,4184782288,1,0,25971,825,26201,0,26200,0,15.784400,35592,286
4,73990908078847,4624,-269540012,1,0,4280170480,2,0,22028,0,13100,0,13100,0,25.900999,38408,309


In [26]:
df_members.shape

(1750072, 17)

In [21]:
df_glossary.head()

,item_cd,item_class,item_name,item_desc2,item_level
0,3553539792,Cruiser,ARP Ashigara,Japan,7
1,3551442640,Cruiser,ARP Haguro,Japan,7
2,3553572560,Battleship,ARP Haruna,Japan,5
3,3552523984,Battleship,ARP Hiei,Japan,5
4,3559143120,Auxiliary,ARP I-401,Japan,11


In [33]:
df_glossary.item_class.unique()

array(['Cruiser', 'Battleship', 'Auxiliary', 'AirCarrier', 'Destroyer'],
      dtype=object)

In [27]:
df_glossary.shape

(324, 5)

In [25]:
df_catalog.cat_value = df_catalog.cat_value.astype('int')
df_catalog.head()

,cat_type,cat_name,cat_value
0,ARENA_TYPES,00_CO_ocean,33
1,ARENA_TYPES,00_pure_gameplay,0
2,ARENA_TYPES,01_NA_escort,27
3,ARENA_TYPES,01_NA_escort_markers,28
4,ARENA_TYPES,01_solomon_islands,21


In [30]:
len(df_arenas.map_type_id.unique())

35

In [29]:
len(set(df_catalog.cat_value.unique()).intersection(set(df_arenas.map_type_id.unique())))

35

In [37]:
df_catalog.cat_type.unique()

array(['ARENA_TYPES', 'TEAM_BUILD_TYPE'], dtype=object)

In [28]:
df_catalog.shape

(78, 3)

In [37]:
df_catalog.cat_type.unique()

array(['ARENA_TYPES', 'TEAM_BUILD_TYPE'], dtype=object)

## Daily Granulation

In [38]:
conn = sqlite3.connect('data/Dataset-ships.db')

query_daily_granulation = '''SELECT
  DATE(start_dt) AS date,
  cat_type AS team_build_type,
  item_name AS ship_name,
  item_class AS ship_class,
  item_level AS ship_level,
  map_type_id,
    COUNT(*) AS TotalBattles,
    SUM(CASE WHEN Winner_team_id = 0 THEN 1 ELSE 0 END) AS Team1Wins,
    SUM(CASE WHEN Winner_team_id = 1 THEN 1 ELSE 0 END) AS Team2Wins,
    AVG(Duration_sec) AS AvgBattleDuration,
    SUM(Ships_killed) AS TotalShipsKilled,
    SUM(Damage) AS TotalDamageCaused,
    SUM(Received_damage) AS TotalReceivedDamage,
    SUM(Credits) AS TotalCreditsGained,
    SUM(Exp) AS TotalExperienceGained,
    1.0 * SUM(CASE WHEN Winner_team_id = Team_id THEN 1 ELSE 0 END) / COUNT(*) AS Winrate,
    1.0 * SUM(CASE WHEN Is_alive = 1 THEN 1 ELSE 0 END) / COUNT(*) AS Survivability,
    SUM(Damage) / COUNT(*) AS AvgDamagePerBattle,
    1.0 *SUM(Ships_killed) / NULLIF(SUM(CASE WHEN Received_damage > 0 THEN 1 ELSE 0 END), 0) AS FragsToDeathsRatio,
    AVG(Credits) AS AvgCreditsIncome,
    AVG(Exp) AS AvgExperienceIncome
FROM arenas a
JOIN arena_members m ON a.arena_id = m.arena_id
JOIN glossary_ships g ON m.vehicle_type_id = g.item_cd
JOIN catalog_items c ON CAST(c.cat_value as INT) = a.map_type_id
GROUP BY date, team_build_type, ship_name, ship_class, ship_level, map_type_id;'''

df_daily_gran = pd.read_sql_query(query_daily_granulation, conn)

conn.close()

In [39]:
df_daily_gran

,date,team_build_type,ship_name,ship_class,ship_level,map_type_id,TotalBattles,Team1Wins,Team2Wins,AvgBattleDuration,...,TotalDamageCaused,TotalReceivedDamage,TotalCreditsGained,TotalExperienceGained,Winrate,Survivability,AvgDamagePerBattle,FragsToDeathsRatio,AvgCreditsIncome,AvgExperienceIncome
0,2017-09-01,ARENA_TYPES,ARP Ashigara,Cruiser,7,1,2,2,0,634.000000,...,58178,66582,224182,2432,1.000000,0.500000,29089,0.000000,112091.000000,1216.000000
1,2017-09-01,ARENA_TYPES,ARP Ashigara,Cruiser,7,3,4,4,0,767.000000,...,91487,131626,123807,795,0.500000,0.250000,22871,0.750000,30951.750000,198.750000
2,2017-09-01,ARENA_TYPES,ARP Ashigara,Cruiser,7,5,2,2,0,537.000000,...,33452,70994,61199,490,0.500000,0.500000,16726,0.500000,30599.500000,245.000000
3,2017-09-01,ARENA_TYPES,ARP Ashigara,Cruiser,7,9,11,3,8,528.909091,...,311835,291115,497573,4571,0.545455,0.454545,28348,0.272727,45233.909091,415.545455
4,2017-09-01,ARENA_TYPES,ARP Ashigara,Cruiser,7,40,1,0,1,792.000000,...,29181,39202,76736,646,0.000000,0.000000,29181,0.000000,76736.000000,646.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115107,2017-09-30,TEAM_BUILD_TYPE,Zao,Cruiser,10,2,38,19,19,758.210526,...,2649384,1655160,4874777,37143,0.526316,0.289474,69720,0.894737,128283.605263,977.447368
115108,2017-09-30,TEAM_BUILD_TYPE,Zao,Cruiser,10,4,24,17,7,796.666667,...,1530999,822059,2724494,20719,0.500000,0.541667,63791,0.590909,113520.583333,863.291667
115109,2017-09-30,TEAM_BUILD_TYPE,Zao,Cruiser,10,5,44,21,23,747.454545,...,2657369,1618174,4782527,34965,0.477273,0.522727,60394,0.837209,108693.795455,794.659091
115110,2017-09-30,TEAM_BUILD_TYPE,Zuiho,AirCarrier,5,3,40,19,21,817.200000,...,1664259,632692,2441402,22206,0.475000,0.625000,41606,1.545455,61035.050000,555.150000


## View Dataset-purchases

In [6]:
conn = sqlite3.connect('data/Dataset-purchases.db')

query_tables = "SELECT * from sqlite_master WHERE type='table'"

df_tables = pd.read_sql_query(query_tables, conn)

conn.close()

In [7]:
df_tables

,type,name,tbl_name,rootpage,sql
0,table,packs_purchases,packs_purchases,1482,"CREATE TABLE packs_purchases(\n purchaser_id,..."


In [15]:
conn = sqlite3.connect('data/Dataset-purchases.db')

query_purchase = "SELECT * from packs_purchases"

df_purchase = pd.read_sql_query(query_purchase, conn)

conn.close()

In [18]:
df_purchase.head()

,purchaser_id,purchase,purchase_dt
0,-1000614515285282804,exclusive_pack,2024-06-12 15:59:40
1,-1000614515285282804,exclusive_pack,2024-06-12 16:02:16
2,-1001734683879252035,common_pack,2024-06-10 10:14:51
3,-10018688817535170,common_pack,2024-06-09 17:48:30
4,-10018688817535170,common_pack,2024-06-10 02:38:09


## Purchase Patterns

In [66]:
conn = sqlite3.connect('data/Dataset-purchases.db')

query_pattern = '''SELECT
    pattern,
    COUNT(DISTINCT purchaser_id) AS user_count
FROM (
    SELECT
        p1.purchaser_id,
        GROUP_CONCAT(p2.purchase, ' - ') AS pattern
    FROM packs_purchases p1
    LEFT JOIN packs_purchases p2 ON p1.purchaser_id = p2.purchaser_id AND p2.purchase_dt >= p1.purchase_dt
    GROUP BY p1.purchaser_id, p1.purchase_dt
) patterns
GROUP BY pattern
ORDER BY user_count DESC;
'''

df_pattern = pd.read_sql_query(query_pattern, conn)
conn.close()

In [67]:
df_pattern

,pattern,user_count
0,common_pack,33494
1,exclusive_pack,16666
2,common_pack - common_pack,14014
3,common_pack - common_pack - common_pack,7034
4,common_pack - common_pack - common_pack - comm...,4083
...,...,...
131,common_pack - common_pack - common_pack - comm...,1
132,common_pack - common_pack - common_pack - comm...,1
133,common_pack - common_pack - common_pack - comm...,1
134,common_pack - common_pack - common_pack - comm...,1
